# Trabalho Prático 1

## Introdução

A resolução deste trabalho prático tem como objetivo implementar um abiente seguro, constituído por um emissor e um recetor, através de sessões cifradas e seguras.
A solução implmentada, foi estruturada da seguinte forma:

- Construção de uma sessão de comunicação síncrona entre um emissor e um recetor.
- Utilização do protocolo de **Diffie_Hellman** com verificação de chave e autenticação dos agentes, na sessão anteriormente referida.
- Utilização do **DSA** para a autenticação dos agentes.
- Utilização do **TAES** como cifra simétrica, com autenticação do criptograma em cada superbloco.
- Implementar novamente o cenário anterior mas, desta feita vez, utilizando o **ECDH** (Elliptic Curve Diffie-Hellman) em vez do **DH** e o **ECDSA** (Elliptic Curve Digital Signature Algorithm) em vez do **DSA**.

Este relatório está escrito de forma a que o texto escrito entre o código desenvolvido seja suficientemente explicativo do que está a ser implementado, o que, na nossa opinião, permite uma leitura e compreensão facilitada do mesmo.

## Sessão Síncrona com _Diffie-Hellman_

### Imports

De seguida, encontram-se os módulos **Python** importados e necessários para desenvolver a sessão síncrona entre os agentes, utilizando o protocolo **DH** e o **DSA**.

In [1]:
import os,io
from multiprocessing import Process,Pipe
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.exceptions import *
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

### Definição das funções do processo criptográfico

Esta secção tem o objetivo de definir funções que serão utilizadas nas sessões síncronas, tanto na que utiliza **DH** como na que utiliza **ECDH**. Essas funções são as seguintes:

- `Hash(s)`, que tem como propósito calcular um _digest_ de uma dada mensagem.

- `next_tweak(initial,current_counter)`, que tem como propósito calcular o próximo _tweak_ a utilizar no bloco que vai ser cifrado/decifrado, com base no _nounce_ inicial e no número do bloco que está a ser processadoa atualmente.

- `bytes_xor(a,b)`, que tem como propósito realizar o _XOR_ _byte_ a _byte_ de duas dadas sequências de _bytes.

- `calculate_parity(blocks,number_blocks)`, que tem como objetivo calcular a paridade de um conjunto de blocos, para que seja gerada a sua _tag_ de autenticação.

- `calculate_auth_tweak(initial_tweak,number_blocks)`, que tem como objetivo calcular o _tweak_ utilizado para cifrar a paridade dos blocos cifrados.

- As funções `cifra_blocos(current_counter,initial_tweak,blocks,number_blocks,cipher_context)` e `decifra_blocos(current_counter,initial_tweak,blocks,number_blocks,decipher_context)` têm como objetivo cifrar/decifrar, respetivamente, um conjunto de blocos aplicando a cada bloco a função de cifra ou de decifragem.

- As funções `cifra(block,tweak,cipher_context)` e `decifra(block,tweak,decipher_context` têm como objetivo cifrar/decifrar um dado bloco utilizando para o efeito a função $ Ẽ(s,x) = E(s,x \oplus h(w)) \oplus h(w) $.

É relevante referir que as operações de cifra e de decifra utilizam exatamente a mesma fórmula e, logo, são iguais, mas estão separadas apenas por uma questão de mais simples compreensão do problema em questão, bem como da sua resolução.
Além disso, neste processo criptográfico foi utilizado o modo **TAE** (_Tweakable Authenticated Encryption_)

In [2]:
def Hash(s):
    digest = hashes.Hash(hashes.SHA256(),backend=default_backend())
    digest.update(s)
    return digest.finalize()

def next_tweak(initial, current_counter):
    str = format(current_counter, '063b')
    str = str + '0'
    b_str = int(str,2).to_bytes((len(str) + 7) // 8, 'big')
    return (initial + b_str)

def bytes_xor(a, b) :
    return bytes(x ^ y for x, y in zip(a, b))

def calculate_parity(blocks,number_blocks):
    str = format(0,'128b') # O número 0 é elemento neutro no XOR
    parity = int(str,2).to_bytes((len(str) + 7) // 8, 'big')
    bytes_processed = 0
    while(bytes_processed < (16*number_blocks)):
        max_bytes = bytes_processed + 16
        block_to_xor = blocks[bytes_processed:max_bytes]
        parity = bytes_xor(parity,block_to_xor)
        bytes_processed += 16
    return parity

def calculate_auth_tweak(initial_tweak,number_blocks):
    str = format(number_blocks*16,'063b')
    str = str + '1'
    b_str = int(str,2).to_bytes((len(str) + 7) // 8, 'big')
    return (initial_tweak + b_str)

def cifra_blocos(current_counter,initial_tweak,blocks,number_blocks,cipher_context):
    bytes_processed = 0
    cryptogram = bytearray(160)
    while(bytes_processed < (16*number_blocks)):
        max_bytes = bytes_processed + 16
        block_to_process = blocks[bytes_processed:max_bytes]
        tweak = next_tweak(initial_tweak,current_counter)
        block_crypt = cifra(block_to_process,tweak,cipher_context)
        cryptogram[bytes_processed:max_bytes] = block_crypt
        bytes_processed += 16
        current_counter += 1
    return {"crypt": bytes(cryptogram), "i": current_counter}
        

def cifra(block,tweak,cipher_context):
    hw = Hash(tweak)
    x_xor_hw = bytes_xor(block,hw)
    e_s_x_hw = cipher_context.update(x_xor_hw)
    cryptogram = bytes_xor(e_s_x_hw, hw)
    return cryptogram

def decifra_blocos(current_counter,initial_tweak,blocks,number_blocks,decipher_context):
    bytes_processed = 0
    text = bytearray(160)
    while(bytes_processed < (16*number_blocks)):
        max_bytes = bytes_processed + 16
        block_to_process = blocks[bytes_processed:max_bytes]
        tweak = next_tweak(initial_tweak,current_counter)
        block_text = decifra(block_to_process,tweak, decipher_context)
        text[bytes_processed:max_bytes] = block_text
        bytes_processed += 16
        current_counter += 1
    return {"text": bytes(text), "i": current_counter}

def decifra(block,tweak,decipher_context):
    hw = Hash(tweak)
    x_xor_hw = bytes_xor(block,hw)
    e_s_x_hw = decipher_context.update(x_xor_hw)
    cryptogram = bytes_xor(e_s_x_hw, hw)
    return cryptogram
    

### Definição da classe de multiprocessamento

Embaixo definem-se a classe de multiprocessamento, que permite uma comunicação bidireccional com o _Emitter_ e o _Receiver_, sendo estes dois processos criados e implementados pela **API** [**multiprocessing**](https://docs.python.org/2/library/multiprocessing.html).

In [3]:
class BiConnection(object):
    def __init__(self,left,right):
        left_side,right_side = Pipe()
        self.timeout = None
        self.left_process = Process(target=left,args=(left_side,))
        self.right_process = Process(target=right,args=(right_side,))
        self.left = lambda : left(left_side)
        self.right = lambda : right(right_side)
    
    def auto(self,proc=None):
        if proc == None:
            self.left_process.start()
            self.right_process.start()
            self.left_process.join(self.timeout)
            self.right_process.join(self.timeout)
        else:
            proc.start()
            proc.join()
    
    def manual(self): 
        self.left()
        self.right()

### Geração dos parâmetros _Diffie-Hellman_

A próxima secção implementa a geração dos parâmetros necessários para a derivação de chaves **DH** e **DSA**, por parte de ambos os agentes.

In [4]:
parameters_dh = dh.generate_parameters(generator=2,key_size=1024,backend=default_backend())
# O tamanho tem que ser 3072 bits, alterar depois.

### Geração das chaves permanentes dos agentes

Esta secção tem como objetivo gerar as chaves permanentes dos agentes envolvidos na comunicação, tendo em conta que estas são conhecidas por ambos, e serão aplicadas na assinatura digital das mensagens.

In [5]:
# Geração das chaves do Emissor
emitter_dsa_sk = dsa.generate_private_key(3072,default_backend()) #chave privada DSA
emitter_dsa_pk = emitter_dsa_sk.public_key() #chave pública DSA

# Geração das chaves do Recetor
receiver_dsa_sk = dsa.generate_private_key(3072,default_backend()) #chave privada DSA
receiver_dsa_pk = receiver_dsa_sk.public_key() #chave pública DSA

### Definição do Emissor e do Recetor

Por fim definiu-se o comportamento de cada um dos agentes nesta sessão síncrona segura de troca de informação. Na definição de ambos, é assumido que as chaves públicas **DSA** utilizadas na autenticação dos agentes já são conhecidas por ambos. As chaves **DH** são geradas por sessão de modo a acordar uma chave temporária, para que seja possível cifrar e decifrar uma mensagem utilizando uma primitiva simétrica.


In [6]:
def Emitter(connection):
    #Implementação Protocolo Diffie-Hellman
    emitter_dh_sk = parameters_dh.generate_private_key()
    emitter_dh_pk = emitter_dh_sk.public_key()
    pub = emitter_dh_pk.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo)
    signature = emitter_dsa_sk.sign(pub,hashes.SHA256())
    message = {'pk': pub, 'sig': signature}
    connection.send(message)
    receiver_first = connection.recv()
    receiver_dh_public_key = serialization.load_pem_public_key(receiver_first['pk'],default_backend())
    receiver_dsa_pk.verify(receiver_first['sig'],receiver_first['pk'],hashes.SHA256())
    master_key = emitter_dh_sk.exchange(receiver_dh_public_key)
    
    #Passar a chave de sessão acordada (master_key) por um kdf
    shared_key = HKDF(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = None,
        info = b'exchange data',
        backend = default_backend()
    ).derive(master_key)
    
    message_size = 1600
    gen_bytes = os.urandom(message_size)
    print('gen_bytes')
    print(gen_bytes)
    inputs = io.BytesIO(gen_bytes)
    #inicialização do tweak como um nounce de 64 bits (n/2)
    tweak_init = os.urandom(8)
    
    connection.send(tweak_init)
    
    buffer = bytearray(160)
    cipher = Cipher(algorithms.AES(shared_key),modes.ECB(),backend=default_backend()).encryptor()
    i = 1
    try:
        while inputs.readinto(buffer):
            cipher_object = cifra_blocos(i,tweak_init,buffer,10,cipher)
            cryptogram = cipher_object["crypt"]
            parity = calculate_parity(buffer,10)
            auth_tweak = calculate_auth_tweak(tweak_init,10)
            auth_tag = cifra(parity,auth_tweak,cipher)
            i = cipher_object["i"]
            msg_to_send = {'crypt': cryptogram, 'tag': auth_tag}
            connection.send(msg_to_send)
            msg = connection.recv()
            if(not(msg == 'OK')):
                print('Erro no processo!')
                break
    except Exception as err:
        print('Erro no emissor! {0}'.format(err))
    connection.send('finalized')   
    inputs.close()
    connection.close()    

In [7]:
def Receiver(connection):
    #Implementação protocolo Diffie-Hellman
    emitter_first = connection.recv()
    receiver_dh_sk = parameters_dh.generate_private_key()
    receiver_dh_pk = receiver_dh_sk.public_key()
    emitter_dh_public_key = serialization.load_pem_public_key(emitter_first['pk'],default_backend())
    emitter_dsa_pk.verify(emitter_first['sig'],emitter_first['pk'],hashes.SHA256())
    master_key = receiver_dh_sk.exchange(emitter_dh_public_key)
    pub = receiver_dh_pk.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo)
    signature = receiver_dsa_sk.sign(pub,hashes.SHA256())
    message = {'pk':pub,'sig':signature}
    connection.send(message)
    
    #Passar a chave de sessão acordada (master_key) por um kdf
    shared_key = HKDF(
        algorithm = hashes.SHA256(),
        length = 32,
        salt = None,
        info = b'exchange data',
        backend = default_backend()
    ).derive(master_key)
    
    outputs = io.BytesIO()
    tweak_init = connection.recv()
    
    cipher = Cipher(algorithms.AES(shared_key),modes.ECB(),backend=default_backend()).decryptor()
    i = 1
    try:
        while True:
            buffer = connection.recv()
            if(buffer == 'finalized'):
                outputs.write(cipher.finalize())
                break
            else:
                crypt = buffer['crypt']
                tag = buffer['tag']
                decipher_object = decifra_blocos(i, tweak_init,crypt,10,cipher)
                text = decipher_object["text"]
                i = decipher_object["i"]
                parity = calculate_parity(text,10)
                auth_tweak = calculate_auth_tweak(tweak_init,10)
                auth_tag = decifra(tag,auth_tweak,cipher)
                if(auth_tag == parity):
                    connection.send('OK')
                else:
                    print('AUTH ERROR')
                    connection.send('AUTH ERROR')
                    break
                outputs.write(text)
        print('decrypted:')
        print(outputs.getvalue())
    except Exception as Err:
        print('Erro no recetor! {0}'.format(Err))
        
    outputs.close()
    connection.close()
    

In [8]:
BiConnection(Emitter,Receiver).auto()

gen_bytes
b'\x8e\xe9\x8f\x90Vs)\xe4\xdb\xb7\xee\x96\xc0L_\xffy`\x0f{\xf0\x8d\x8a\xe1^\xebjS\xa3\xder\xb1y8\xeb\x93\xea-7\x87\xe7\xe7:C\xa2\x97\xac@F\xea>C\x8a6^\x81\x92\xd2xe/D\t\x10\xf7\xc4Us\xaa\xd9\x8c\xc3l\x1f\xc8\xcb,%W/\x8a\x95\xcbi9>\x8e\x1e"\xaf\xad\x98\x1c\xfb)B\xca\xfbnw\xbe\x07\xc6\xad\x10\xbd\n\xab!\xb1\xe7s \x8f\xf1h\xddn\xeb\xfc\xe2\x1br\xd7Q\xfa\xe5}\xb1\xab\x07\xf7zf\x9a\x84-=\xf5P#\xdf\x1f\xe0g36e\x97\xd1\xb9x\x8f#\x13n\xc5K\xcb\xa69\x8d9;\xdd*v\xe3|\x85\x1c\xdbp\xfcUK\xb8\x81\x92\x98\xde\x08\xffB\x7f\xd8\xdd\x85:9\xc8\x82<\x9d#n\xd1\xff\x15\xdd\xf4\xf4Y\x9fL\x9a\x0f\xe7=H\x93\xb5\x0e/\xa2\xc2\x9b\x183E!k\xc5\xde\x0c\xd6`\xc3M\xf4\x94\x8c)\x87&\xae\xe5x\x0fo\xc8\x05\x00\xf5\x00\xdc\x1a\x07\x1b\x14*\xf3_\xdf@\x97\x10\xff\xd6\xbe\xbd+p\xaa\xcfRpau\x10q\x17\x18-\x90\xd76R#\x8b\xb9\xa1\xad\xffb.+\xc5\xa5A\xfd\xc4\x81>\x1c?-\xd9\x1a\xbd\xf7\xd5\t<L\x85~\xe1\t\x83\xa60V\xa4*\x99\x87U\xba\xf18@\x82\x85<\x19\xbc\x1b\x04@\xb9\x14\xd9\x05v\xff\xb9\xa8\xde\x1c\x110\xf2W\x1eh\x0b\

## Sessão Síncrona com _Elliptic Curve Diffie-Hellman_